### Part 1.7 - Extract_data_35000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Seventh 5000 movies (35,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the seventh 5000 movies and create the souplist of those URLs
"""
content_seven = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_seven_13012020.pkl'), 'rb') as f:
    
    content_seven = pickle.load(f)

print("Number of URLs: {}".format(len(content_seven)))

content_souplist_seven = []

for i in tqdm_notebook(content_seven):
    
    content_souplist_seven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_seven)))

In [ ]:
"""
Open the synopsis urls of the seventh 5000 movies and create the souplist of those URLs
"""
synopsis_seven = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_seven_13012020.pkl'), 'rb') as f:
    
    synopsis_seven = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_seven)))

synopsis_souplist_seven = []

for i in tqdm_notebook(synopsis_seven):
    
    synopsis_souplist_seven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_seven)))

In [ ]:
"""
Open the reviews urls of the seventh 5000 movies and create the souplist of those URLs
"""
review_seven = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_seven_15012020.pkl'), 'rb') as f:
    
    review_seven = pickle.load(f)

print("Number of URLs: {}".format(len(review_seven)))

review_souplist_seven = []

for i in tqdm_notebook(review_seven):
    
    review_souplist_seven.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_seven)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_seven)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_seven)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_seven)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_seven)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_seven)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_seven)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_seven_13012020.pkl'), 'rb') as f:
    
    content_seven = pickle.load(f)

print("Number of URLs: {}".format(len(content_seven)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_seven_15012020.pkl'), 'rb') as f:
    
    review_seven = pickle.load(f)

print("Number of URLs: {}".format(len(review_seven)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\plot_seven_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\rating_seven_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\actors_seven_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\director_seven_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\synopsis_seven_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\pre-indexed files\\reviews_seven_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[44,79,170,176,233,237,315,328,347,638,831,855,870,938,952,1115,1125,1176,1600,1611,1619,1620,1705,1868,1940,2183,2245,2277,2322,2337,2382,2402,2420,2429,2525,2534,2569,2581,2714,2957,3332,3622,3898,4058,4063,4107,4251,4267,4309,4334,4489,4524,4673,4675,4676,4677,4681,4690,4699,4703,4711,4880,1687,2402,2405,2527,2528,2561,2569,3825,4524,938,2402,2569,36,52,88,93,107,149,174,227,323,395,589,621,623,631,635,636,640,641,652,655,664,670,846,847,966,967,1076,1079,1102,1122,1238,1249,1256,1266,1310,1408,1526,1527,1571,1576,1687,1951,1952,1972,2043,2115,2121,2156,2183,2198,2250,2256,2325,2341,2350,2351,2356,2377,2386,2389,2494,2502,2539,2582,2708,2743,2799,2817,2829,2870,2889,2892,2894,3068,3069,3070,3084,3089,3123,3130,3323,3325,3380,3438,3581,3723,3732,3733,3741,3748,3751,3756,3758,3760,3761,3768,3772,3776,3790,3791,3807,3808,3823,3899,3976,4023,4037,4061,4089,4090,4092,4098,4147,4286,4335,4438,4439,4472,4499,4667,4676,4680,4731,4801,4855,4865,4866,4890,4902]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_seven = [i for j, i in enumerate(content_seven) if j not in index_remove]
review_index_seven = [i for j, i in enumerate(review_seven) if j not in index_remove]
print(len(content_index_seven))
print(len(review_index_seven))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_seven\\content_index_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(content_index_seven, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_seven\\review_index_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_seven, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_seven = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\content_index_seven_20012020.pkl'), 'rb') as f:
    
    content_index_seven = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_seven)))

for i in tqdm_notebook(content_index_seven):
    
    content_souplist_seven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_seven)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\review_index_seven_20012020.pkl'), 'rb') as f:
    
    review_seven = pickle.load(f)

print("Number of URLs: {}".format(len(review_seven)))

review_souplist_seven = []

for i in tqdm_notebook(review_seven):
    
    review_souplist_seven.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_seven)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_seven)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_seven)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_seven)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_seven)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_seven)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\plot_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\actors_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\director_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\reviews_seven_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_seven, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\rating_seven_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[44,79,170,176,233,237,315,328,347,638,831,855,870,938,952,1115,1125,1176,1600,1611,1619,1620,1705,1868,1940,2183,2245,2277,2322,2337,2382,2402,2420,2429,2525,2534,2569,2581,2714,2957,3332,3622,3898,4058,4063,4107,4251,4267,4309,4334,4489,4524,4673,4675,4676,4677,4681,4690,4699,4703,4711,4880,1687,2402,2405,2527,2528,2561,2569,3825,4524,938,2402,2569,36,52,88,93,107,149,174,227,323,395,589,621,623,631,635,636,640,641,652,655,664,670,846,847,966,967,1076,1079,1102,1122,1238,1249,1256,1266,1310,1408,1526,1527,1571,1576,1687,1951,1952,1972,2043,2115,2121,2156,2183,2198,2250,2256,2325,2341,2350,2351,2356,2377,2386,2389,2494,2502,2539,2582,2708,2743,2799,2817,2829,2870,2889,2892,2894,3068,3069,3070,3084,3089,3123,3130,3323,3325,3380,3438,3581,3723,3732,3733,3741,3748,3751,3756,3758,3760,3761,3768,3772,3776,3790,3791,3807,3808,3823,3899,3976,4023,4037,4061,4089,4090,4092,4098,4147,4286,4335,4438,4439,4472,4499,4667,4676,4680,4731,4801,4855,4865,4866,4890,4902]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_seven = dataset.iloc[30000:35000].reset_index(drop=True)
dataset_seven = dataset_seven[~dataset_seven.index.isin(index_remove)]
dataset_seven.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the seventh mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\plot_seven_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\rating_seven_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\actors_seven_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\director_seven_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_seven\\reviews_seven_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_seven['actors'] = actors
dataset_seven['plot'] = plot
dataset_seven['imdb_rating'] = rating
dataset_seven['director'] = director
dataset_seven['reviews'] = reviews
dataset_seven = dataset_seven.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[7,19,35,58,80,91,93,114,141,146,155,162,163,165,188,211,212,213,215,221,246,251,296,299,303,305,357,367,417,431,437,534,535,549,574,614,619,625,627,639,666,671,675,679,722,770,793,794,799,800,847,890,902,981,990,1026,1030,1033,1034,1036,1039,1052,1056,1075,1111,1113,1173,1186,1188,1189,1223,1255,1268,1326,1331,1332,1349,1357,1382,1386,1401,1403,1416,1419,1471,1473,1475,1518,1519,1540,1551,1563,1594,1597,1599,1609,1629,1635,1638,1681,1696,1719,1725,1732,1746,1749,1759,1769,1800,1871,1901,1909,1940,1955,2023,2026,2027,2033,2036,2037,2040,2041,2043,2045,2056,2057,2058,2085,2086,2087,2109,2111,2115,2118,2120,2121,2126,2127,2130,2193,2208,2222,2230,2237,2245,2246,2261,2263,2265,2267,2269,2272,2273,2275,2281,2283,2284,2289,2292,2294,2299,2300,2316,2357,2365,2383,2384,2386,2387,2388,2390,2396,2399,2400,2401,2404,2405,2407,2413,2415,2417,2437,2439,2511,2519,2526,2532,2548,2608,2613,2637,2640,2676,2684,2692,2697,2704,2714,2718,2719,2721,2764,2766,2777,2813,2853,2860,2883,2886,2889,2894,2953,2954,2966,2975,3003,3031,3039,3110,3139,3149,3155,3158,3175,3202,3203,3210,3230,3266,3279,3328,3344,3345,3346,3349,3352,3354,3359,3373,3374,3406,3422,3424,3434,3456,3467,3468,3481,3521,3525,3548,3581,3582,3583,3587,3596,3598,3600,3601,3604,3605,3608,3615,3620,3622,3623,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633,3637,3641,3647,3648,3650,3651,3654,3655,3669,3696,3711,3718,3729,3730,3734,3750,3764,3766,3774,3832,3869,3872,3875,3915,3921,3922,3926,3934,3951,3971,3997,4012,4013,4039,4047,4048,4051,4074,4091,4094,4098,4112,4122,4148,4165,4168,4196,4202,4212,4215,4236,4237,4240,4242,4243,4247,4248,4249,4250,4251,4252,4254,4257,4258,4263,4270,4281,4284,4290,4303,4317,4321,4326,4327,4334,4336,4339,4350,4361,4383,4404,4412,4485,4495,4516,4517,4521,4550,4641,4646,4650,4671,4681,4691,4692,4695,4699,4700,4705,4710,4720,4721,4800]
dataset_seven = dataset_seven[~dataset_seven.index.isin(index_remove)]
dataset_seven = dataset_seven[dataset_seven.astype(str)['reviews'] != '[]']
dataset_seven.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_seven.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_seven\\dataset_seven_final_25012020.pkl"))